In [0]:
# !pip install mlflow
import mlflow
from databricks import automl
from pyspark.sql import functions as F

In [0]:
CATALOG = "lab"
SCHEMA = "data"

spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {SCHEMA}")

# Build a labeled training set from silver tables
clicks = spark.table("clicks_silver").select("user_id", "event_date")
orders = spark.table("orders_silver").select("user_id", "order_date")

label = (
    clicks.join(
        orders,
        (clicks.user_id == orders.user_id) & (clicks.event_date == orders.order_date),
        "left"
    )
    .withColumn(
        "converted",
        F.when(F.col("order_date").isNotNull(), F.lit(1)).otherwise(F.lit(0))
    )
    .select(
        clicks["user_id"],
        clicks["event_date"].alias("date"),
        "converted"
    )
)

display(label)

In [0]:
features = (
    spark.table("impressions_silver")
    .groupBy("user_id", "event_date")
    .agg(F.count("*").alias("impressions"))
    .join(
        spark.table("clicks_silver")
        .groupBy("user_id", "event_date")
        .agg(F.count("*").alias("clicks")),
        ["user_id", "event_date"],
        "left"
    )
    .withColumnRenamed("event_date", "date")
)

display(features)

In [0]:
train_df = label.join(features, ["user_id", "date"], "left")
display(train_df)

In [0]:
summary = automl.classify(
    train_df,
    target_col="converted",
    experiment_dir=f"/propensity_model",
    timeout_minutes=10
)

print("Best trial run_id:", summary.best_trial.mlflow_run_id)
print("Best model URI:", summary.best_trial.model_path)